In [1]:
import json
import urllib.request
import rdflib
from os import listdir
from SPARQLWrapper import SPARQLWrapper, N3

In [2]:
ttlFolder='../data/ttl/main/'
ttlOutput='../data/ttl/additional/wd.ttl'
batchSizeForRetrieval = 100
wdEndpoint = "https://query.wikidata.org/sparql"

In [3]:
inputFiles = []
for file in listdir(ttlFolder):
    if file.endswith('.ttl'):
        inputFiles.append(file)

In [4]:
wdIdentifiers = []
for file in inputFiles:
    g = rdflib.Graph()
    g.parse(ttlFolder + file, format='ttl')
    queryResults = g.query(
    """SELECT DISTINCT ?wd WHERE {
        ?s ?p ?wd .
        FILTER(REGEX(STR(?wd),"http://www.wikidata.org/entity/"))
    }""")
    for row in queryResults:
        wdIdentifiers.append(str(row[0]))
wdIdentifiers = list(set(wdIdentifiers))

In [5]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

with open(ttlOutput, 'w') as outputFile:
     outputFile.write('')

sparql = SPARQLWrapper(wdEndpoint)    

with open(ttlOutput, 'ab') as outputFile:
    for batch in chunker(wdIdentifiers, batchSizeForRetrieval):
        query = """
             PREFIX wdt: <http://www.wikidata.org/prop/direct/>
             CONSTRUCT {
                 ?entity wdt:P625 ?coordinates .
             } WHERE {
                 ?entity wdt:P625 ?coordinates .
                 VALUES (?entity) {
                     %s
                 }
             }

        """ % ( "(<" + ">)\n(<".join(batch) + ">)" )
        sparql.setQuery(query)
        results = sparql.query().convert()
        outputFile.write(results.serialize(format='turtle'))